<a href="https://colab.research.google.com/github/wonjae124/Capston/blob/colab/wide_deep_base_tutorial(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import torch
import pandas as pd
import numpy as np

from torch import nn

In [19]:
df = pd.DataFrame({"color": ["r", "b", "g"], "size": ["s", "n", "l"]})
df.head()

,color,size
0,r,s
1,b,n
2,g,l


In [20]:
obs_0_oh = (np.array([1.0, 0.0, 0.0, 1.0, 0.0, 0.0])).astype("float32")
obs_0_oh

array([1., 0., 0., 1., 0., 0.], dtype=float32)

In [21]:
obs_0_le = (np.array([0, 3])).astype("int64")
obs_0_le

array([0, 3])

In [22]:
lin = nn.Linear(6, 1)
lin

Linear(in_features=6, out_features=1, bias=True)

In [23]:
emb = nn.Embedding(6, 1)
emb

Embedding(6, 1)

In [24]:
emb.weight = nn.Parameter(lin.weight.reshape_as(emb.weight))
emb

Embedding(6, 1)

In [8]:
x= lin(torch.tensor(obs_0_oh))
x

tensor([0.1860], grad_fn=<AddBackward0>)

In [9]:
y = emb(torch.tensor(obs_0_le)).sum() + lin.bias
y

tensor([0.1860], grad_fn=<AddBackward0>)

In [10]:
!pip install pytorch-widedeep
!pip install git+https://github.com/jrzaurin/pytorch-widedeep.git
!git clone https://github.com/jrzaurin/pytorch-widedeep
!cd pytorch-widedeep

     |████████████████████████████████| 21.1 MB 1.2 MB/s 
     |████████████████████████████████| 398 kB 74.0 MB/s 
     |████████████████████████████████| 1.4 MB 51.9 MB/s 
     |████████████████████████████████| 52.7 MB 71 kB/s 
     |████████████████████████████████| 134 kB 55.7 MB/s 
     |████████████████████████████████| 1.6 MB 54.3 MB/s 
     |████████████████████████████████| 4.1 MB 49.0 MB/s 
     |████████████████████████████████| 225 kB 50.0 MB/s 
     |████████████████████████████████| 125 kB 59.3 MB/s 
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.44.0
    Uninstalling grpcio-1.44.0:
      Successfully uninstalled grpcio-1.44.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
  Cloning https://github.com/jrzaurin/pytorch-widedeep.g

In [11]:
import pytorch_widedeep
from pytorch_widedeep.models import Wide


In [12]:
wide = Wide(input_dim=10, pred_dim=1)
wide

Wide(
  (wide_linear): Embedding(11, 1, padding_idx=0)
)

In [13]:
from pytorch_widedeep.models import TabMlp


In [14]:
colnames = ["a", "b", "c", "d", "e"]
cat_embed_input = [(u,i,j) for u, i,  j in zip(colnames[:4], [4]*4, [8]*4)] # 곱하기의 의미로는 4번을 반복함을 뜻 한다
cat_embed_input

[('a', 4, 8), ('b', 4, 8), ('c', 4, 8), ('d', 4, 8)]

In [15]:
column_idx = {k: v for v, k in enumerate(colnames)}  # set을 만들어내는(순서 없이) 중괄호 연산자
column_idx

{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4}

In [16]:
tabmlp = TabMlp(
    column_idx = column_idx, #4개의 임베딩 레이어를 구성함. column_idx는 layer의 이름과 그 순서를 결정한다. 그렇다면 나도 순서를 바꾸어보게 좋을까?
    cat_embed_input = cat_embed_input,
    continuous_cols = ['e'],
    mlp_hidden_dims = [8,4],
)
tabmlp

TabMlp(
  (cat_and_cont_embed): DiffSizeCatAndContEmbeddings(
    (cat_embed): DiffSizeCatEmbeddings(
      (embed_layers): ModuleDict(
        (emb_layer_a): Embedding(5, 8, padding_idx=0)
        (emb_layer_b): Embedding(5, 8, padding_idx=0)
        (emb_layer_c): Embedding(5, 8, padding_idx=0)
        (emb_layer_d): Embedding(5, 8, padding_idx=0)
      )
      (embedding_dropout): Dropout(p=0.1, inplace=False)
    )
    (cont_norm): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (tab_mlp): MLP(
    (mlp): Sequential(
      (dense_layer_0): Sequential(
        (0): Dropout(p=0.1, inplace=False)
        (1): Linear(in_features=33, out_features=8, bias=True)
        (2): ReLU(inplace=True)
      )
      (dense_layer_1): Sequential(
        (0): Dropout(p=0.1, inplace=False)
        (1): Linear(in_features=8, out_features=4, bias=True)
        (2): ReLU(inplace=True)
      )
    )
  )
)

In [17]:
from pytorch_widedeep.models import BasicRNN
basic_rnn = BasicRNN(vocab_size = 4, hidden_dim = 4, n_layers = 1, padding_idx = 0, embed_dim = 4) #hidden_dim은 LSTM의 출력값으로 지정 되고. vocab_size는 word_embed layer의 입력값, embed_dim은 word_embed_layer의 출력 사이즈와 LSTM의 입력 사이즈를 결정한다.
basic_rnn

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


BasicRNN(
  (word_embed): Embedding(4, 4, padding_idx=0)
  (rnn): LSTM(4, 4, batch_first=True, dropout=0.1)
  (rnn_mlp): Identity()
)